# 项目：某平台北京2021年二手房价数据

#### 背景描述：某平台北京市二手房信息，用于数据分析、可视化

#### 数据说明：字段：市区、 小区、 户型、 朝向、 楼层、 装修情况、 电梯、 面积(㎡)、 价格(万元)、 年份

#### 问题描述： 
1、北京二手房地理分布 
2、二手房价格分析 
3、二手房面积分布 
4、二手房价格预测 
5、相关性分析

In [1]:
import pandas as pd
import echarts_python as ec
from echarts_python.charts import Map
from echarts_python.options import TitleOpts, VisualMapOpts
from IPython.display import IFrame

C:\Users\林涵\AppData\Local\Temp\ipykernel_10232\2926254626.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


ModuleNotFoundError: No module named 'echarts_python'

导入数据表，赋值为beijing_list

In [ ]:
beijing_list = pd.read_csv("二手房数据.csv",encoding = 'gb18030')
beijing_list.head()

##### 评估数据
###### 查看数据整齐和清洁度
首先复制一个新的表，作为清洗前后区分

In [ ]:
beijing_list1 = beijing_list
beijing_list1

首先去掉去后一列无数据的列

In [ ]:
beijing_list1 = beijing_list1.drop("Unnamed: 11",axis = 1)
beijing_list1

查看表格数据情况

In [ ]:
beijing_list1.describe()

共23677条数据，接着查看缺失数据情况

In [ ]:
beijing_list1.isnull().sum()

电梯项缺失8255条数据

查看缺失数据情况

In [ ]:
beijing_list1[('电梯')].unique()

补充缺失值

In [ ]:
beijing_list1['电梯'] = beijing_list1['电梯'].fillna('未知')
beijing_list1.isnull().sum()

In [ ]:
beijing_list1.drop(beijing_list1[beijing_list1['电梯'] == '81'].index,inplace=True)
beijing_list1.drop(beijing_list1[beijing_list1['电梯'] == '0'].index,inplace=True)
beijing_list1.drop(beijing_list1[beijing_list1['电梯'] == '3室1厅'].index,inplace=True)
beijing_list1.drop(beijing_list1[beijing_list1['电梯'] == '2室1厅'].index,inplace=True)
beijing_list1.drop(beijing_list1[beijing_list1['电梯'] == '光熙家园一期'].index,inplace=True)
beijing_list1.drop(beijing_list1[beijing_list1['电梯'] == '京师吉地'].index,inplace=True)
beijing_list1['电梯'].unique()

In [ ]:
#电梯数据完整，之后查看朝向数据

In [ ]:
beijing_list1['朝向'].unique()

In [ ]:
beijing_list1['朝向'] = beijing_list1['朝向'].str.replace('南西','西南')
beijing_list1['朝向'].unique()

In [ ]:
beijing_list1_df = pd.DataFrame(beijing_list1)
print(beijing_list1_df.columns)

In [ ]:
beijing_list1.groupby('市区').count()

可以看出丰台、朝阳、海淀、昌平的房数最多。

### 可视化
1北京二手房数量地图分布

In [ ]:
g = beijing_list1.groupby('市区')
df_region = g.count()['小区']
region = df_region.index.tolist()
count = df_region.values.tolist()
df_region

In [ ]:
def m1():
    new = [x + '区' for x in region]
    m = (
            Map()
            .add('', [list(z) for z in zip(new, count)], '北京')
            .set_global_opts(
                title_opts=opts.TitleOpts(title='北京市二手房各区分布'),
                visualmap_opts=opts.VisualMapOpts(max_=3000),
            )
        )
    return m

In [ ]:
m = m1()
m.render('北京市二手房各区分布.html')

In [ ]:


# 将文件名替换为您在上一个示例中使用的文件名
file_path = '北京市二手房各区分布.html'

# 使用 IFrame 显示 HTML 文件
IFrame(file_path, width=800, height=600)

In [ ]:
def grid1():
    df_price.values.tolist()
    price = [round(x,2) for x in df_price.values.tolist()]
    bar = (
        Bar()
        .add_xaxis(region)
        .add_yaxis('数量', count,
                label_opts=opts.LabelOpts(is_show=True))
        .extend_axis(
            yaxis=opts.AxisOpts(
                name="价格（万元）",
                type_="value",
                min_=200,
                max_=900,
                interval=100,
                axislabel_opts=opts.LabelOpts(formatter="{value}"),
            )
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title='各城区二手房数量-平均价格柱状图'),
            tooltip_opts=opts.TooltipOpts(
                is_show=True, trigger="axis", axis_pointer_type="cross"
            ),
            xaxis_opts=opts.AxisOpts(
                type_="category",
                axispointer_opts=opts.AxisPointerOpts(is_show=True, type_="shadow"),
            ),
            yaxis_opts=opts.AxisOpts(name='数量',
                axistick_opts=opts.AxisTickOpts(is_show=True),
                splitline_opts=opts.SplitLineOpts(is_show=False),)
        )
    )

    line2 = (
        Line()
        .add_xaxis(xaxis_data=region)
        .add_yaxis(
            
            series_name="价格",
            yaxis_index=1,
            y_axis=price,
            label_opts=opts.LabelOpts(is_show=True),
            z=10
            )
    )

    bar.overlap(line2)
    grid = Grid()
    grid.add(bar, opts.GridOpts(pos_left="5%", pos_right="20%"), is_control_axis_index=True)
    return grid

In [ ]:
from IPython.display import Image

# 将图片路径替换为您要显示的图片的实际路径
image_path = 'path/to/your/image.jpg'

# 使用 Image 函数显示图片
Image(image_path)